# Setup

Dependencies:
- System: python3
- Python: jupyter, numpy, matplotlib, scipy

Example setup for a Ubuntu system (Mac users, maybe `brew` instead of `sudo apt`; Windows users, learn to love [WSL](https://docs.microsoft.com/en-us/windows/wsl/install-win10)):
```
/usr/bin/python3 -m pip install --upgrade pip
pip install --upgrade jupyter numpy matplotlib scipy
jupyter notebook  # from the directory of this notebook
```
Alternatively, view this notebook on [Google Colab](https://colab.research.google.com/github/StanfordASL/AA203-Examples/blob/master/Lecture-9/Zermelo's%20Problem.ipynb).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize, Bounds

## Simultaneous Method (state and control decision variables)

In [ ]:
def zermelo_simultaneous(u_max=1.0, z0=None, verbose=True):

    # Problem parameters
    t_f = 10
    M = 10
    ℓ = 5
    v = 1
    flow = lambda y: 0.35 / (ℓ**2 / 4) * y * (ℓ - y)

    # Discretization
    N = 20
    h = t_f / N

    # Decision variables
    # z = np.concatenate([x, y, u])
    get_x = lambda z: z[:N + 1]
    get_y = lambda z: z[N + 1:-N]
    get_u = lambda z: z[-N:]

    # Set up problem and `minimize`
    eps = 1e-3  # `minimize` can be a bit finicky about constraints, so slightly loosen state/control box constraints
    bounds = Bounds(
        np.concatenate([np.zeros(N + 1), np.zeros(N + 1), -u_max * np.ones(N)]) - eps,
        np.concatenate([M * np.ones(N + 1), ℓ * np.ones(N + 1), u_max * np.ones(N)]) + eps)

    cost = lambda z: h * np.sum(np.square(get_u(z)))

    def constraints(z):
        return np.concatenate([
            get_x(z)[1:] - get_x(z)[:-1] - h * (v * np.cos(get_u(z)) + flow(get_y(z)[:-1])),  # x dynamics constraint
            get_y(z)[1:] - get_y(z)[:-1] - h * v * np.sin(get_u(z)),                          # y dynamics constraint
            get_x(z)[[0, -1]] - np.array([0., M]),                                            # x initial/terminal constraint
            get_y(z)[[0, -1]] - np.array([0., ℓ])                                             # y initial/terminal constraint
        ])

    if z0 is None:
        z0 = np.concatenate([np.ones(N + 1), np.ones(N + 1), 0.5 * u_max * np.ones(N)])
#         z0 = np.concatenate([np.linspace(0, M, N + 1), np.linspace(0, ℓ, N + 1), 0.5 * u_max * np.zeros(N)])
    result = minimize(cost,
                      z0,
                      bounds=bounds,
                      constraints={
                          'type': 'eq',
                          'fun': constraints
                      },
                      options={'maxiter': 1000})
    if verbose:
        print(result)

    return get_x(result.x), get_y(result.x), get_u(result.x)

In [ ]:
x_opt, y_opt, z_opt = zermelo_simultaneous()
t_f, N = 10, 20
h = t_f / N
plt.figure(figsize=(15, 8))
plt.subplot(1, 2, 1)
plt.plot(x_opt, y_opt)
plt.title("Optimal Trajectory", fontsize=24)
plt.subplot(1, 2, 2)
plt.title("Optimal Control", fontsize=24)
plt.plot(np.arange(N) * h, z_opt)

In [ ]:
x_opt, y_opt, z_opt = zermelo_simultaneous(1.0, verbose=False)
x_opt, y_opt, z_opt = zermelo_simultaneous(0.75, np.concatenate([x_opt, y_opt, z_opt]))
plt.figure(figsize=(15, 8))
plt.subplot(1, 2, 1)
plt.plot(x_opt, y_opt)
plt.title("Optimal Trajectory", fontsize=24)
plt.subplot(1, 2, 2)
plt.title("Optimal Control", fontsize=24)
plt.plot(np.arange(N) * h, z_opt)

## Shooting Method (control decision variables only)

In [ ]:
def zermelo_shooting(u_max=1.0, u0=None, verbose=True):

    # Problem parameters
    t_f = 10
    M = 10
    ℓ = 5
    v = 1
    flow = lambda y: 0.35 / (ℓ**2 / 4) * y * (ℓ - y)

    # Discretization
    N = 20
    h = t_f / N
    dynamics = lambda x, u: x + h * np.array([v * np.cos(u) + flow(x[..., 1]), v * np.sin(u)])

    # Set up problem and `minimize`
    eps = 1e-3
    bounds = Bounds(-u_max * np.ones(N), u_max * np.ones(N))  # control constraints

    cost = lambda u: h * np.sum(np.square(u))

    def constraints(u):
        state = np.zeros(2)
        constraint_list = []
        for ui in u:
            state = dynamics(state, ui)
            constraint_list.append(state)                     # state >= 0 (box constraint with below)
            constraint_list.append(np.array([M, ℓ]) - state)  # state <= [M, ℓ]
        constraint_list.append(state - np.array([M, ℓ]))      # terminal state >= [M, ℓ] as well (enforcing equality)
        return np.concatenate(constraint_list)

    if u0 is None:
        u0 = 0.5 * u_max * np.ones(N)
    result = minimize(cost,
                      u0,
                      bounds=bounds,
                      constraints={
                          'type': 'ineq',
                          'fun': constraints
                      },
                      options={'maxiter': 1000})
    if verbose:
        print(result)

    u_opt = result.x
    states = [np.zeros(2)]
    for ui in u_opt:
        states.append(dynamics(states[-1], ui))
    return [state[0] for state in states], [state[1] for state in states], u_opt

In [ ]:
x_opt, y_opt, z_opt = zermelo_shooting(1.0)
plt.figure(figsize=(15, 8))
plt.subplot(1, 2, 1)
plt.plot(x_opt, y_opt)
plt.title("Optimal Trajectory", fontsize=24)
plt.subplot(1, 2, 2)
plt.title("Optimal Control", fontsize=24)
plt.plot(np.arange(N) * h, z_opt)

In [ ]:
x_opt, y_opt, z_opt = zermelo_shooting(0.75)
plt.figure(figsize=(15, 8))
plt.subplot(1, 2, 1)
plt.plot(x_opt, y_opt)
plt.title("Optimal Trajectory", fontsize=24)
plt.subplot(1, 2, 2)
plt.title("Optimal Control", fontsize=24)
plt.plot(np.arange(N) * h, z_opt)